In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
import numpy as np

In [ ]:

# Replace the URL below with the raw URL of your CSV file
url = 'https://raw.githubusercontent.com/armangupta910/Movie-Recommendor-System/main/dataSet/complete_data%20-%20complete_data%20(1).csv.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(url)

# Display the first few rows of the DataFrame
display(df.head())
print(df.shape[0])
df = df.drop(columns=['timestamp_y'])

# Impute missing values in the 'tag' column with "No Tag"
df['tag'] = df['tag'].fillna('No Tag')

display(df.head())


,userId,movieId,rating,timestamp_x,title,genres,tag,timestamp_y
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN,NaN


102677


,userId,movieId,rating,timestamp_x,title,genres,tag
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,No Tag
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,No Tag
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,No Tag
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,No Tag
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,No Tag


In [ ]:
import pandas as pd
grouped_data = df.groupby('title').agg({'rating': 'mean', 'timestamp_x': 'mean', 'genres': 'first'})

grouped_data.reset_index(inplace=True)
display(grouped_data)



,title,rating,timestamp_x,genres
0,(500) Days of Summer (2009),3.857143,1.388917e+09,Comedy|Drama|Romance
1,*batteries not included (1987),3.285714,1.225801e+09,Children|Comedy|Fantasy|Sci-Fi
2,...All the Marbles (1981),2.000000,1.340342e+09,Comedy|Drama
3,...And Justice for All (1979),3.166667,1.144329e+09,Drama|Thriller
4,00 Schneider - Jagd auf Nihil Baxter (1994),4.500000,1.485101e+09,Comedy|Crime
...,...,...,...,...
9714,night Mother (1986),3.000000,1.065028e+09,Drama
9715,xXx (2002),2.770833,1.211263e+09,Action|Crime|Thriller
9716,xXx: State of the Union (2005),2.000000,1.342892e+09,Action|Crime|Thriller
9717,�� nous la libert�� (Freedom for Us) (1931),1.000000,1.033174e+09,Comedy|Musical


In [ ]:
rating_5_count = (grouped_data['rating'] == 5).sum()

print("Count of rows where rating is 5:", rating_5_count)

rating_5_count = (grouped_data['rating'] < 5).sum()

print("Count of rows where rating is below 5:", rating_5_count)

rating_5_count = (grouped_data['rating'] < 4).sum()

print("Count of rows where rating is below 4:", rating_5_count)

rating_5_count = (grouped_data['rating'] < 3).sum()

print("Count of rows where rating is below 3:", rating_5_count)

rating_5_count = (grouped_data['rating'] < 2).sum()

print("Count of rows where rating is below 2:", rating_5_count)

Count of rows where rating is 5: 296
Count of rows where rating is below 5: 9423
Count of rows where rating is below 4: 7437
Count of rows where rating is below 3: 2749
Count of rows where rating is below 2: 637


**RECOMMENDATION SYSTEM JUST BY GENRE AND RATING**

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Concatenate, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Assuming the dataset is stored in a DataFrame named 'movies_df'

def prepare_data(movies_df):
    # Encode categorical variables
    genre_encoder = LabelEncoder()
    movies_df['genre_encoded'] = genre_encoder.fit_transform(movies_df['genres'])

    # Scale numerical features
    scaler = MinMaxScaler()
    movies_df['rating_scaled'] = scaler.fit_transform(movies_df['rating'].values.reshape(-1, 1))

    return movies_df, genre_encoder

def build_model(num_genres):
    # Define input layers
    genre_input = Input(shape=(1,))
    rating_input = Input(shape=(1,))

    # Embedding layer for genres
    genre_embedding = Embedding(input_dim=num_genres, output_dim=10)(genre_input)
    genre_flatten = Flatten()(genre_embedding)

    # Concatenate embedded features
    merged = Concatenate()([genre_flatten, rating_input])

    # Dense layers
    dense1 = Dense(128, activation='relu')(merged)
    output = Dense(1, activation='sigmoid')(dense1)

    # Model
    model = Model(inputs=[genre_input, rating_input], outputs=output)
    model.compile(loss='mse', optimizer='adam')

    return model

def train_model(model, X_train, y_train):
    early_stopping = EarlyStopping(patience=3, restore_best_weights=True)
    model.fit(X_train, y_train, epochs=10, batch_size=64, validation_split=0.2, callbacks=[early_stopping])

def recommend_movies_with_ann(movies_df, genre_encoder, genre_input, min_rating_input, max_rating_input, top_n=5):
    # Load pre-trained model
    model = build_model(len(genre_encoder.classes_))

    # Prepare input data
    X_genre = movies_df['genre_encoded'].values.reshape(-1, 1)
    X_rating = movies_df['rating_scaled'].values.reshape(-1, 1)
    y_train = movies_df['rating_scaled'].values

    train_model(model, [X_genre, X_rating], y_train)

    # Generate recommendations
    predictions = model.predict([X_genre, X_rating])
    movies_df['predicted_rating'] = predictions * 5  # Scale predictions back to original rating scale

    # Filter movies based on user input rating range and genre
    filtered_movies = movies_df[(movies_df['genres'].str.contains(genre_input)) &
                                (movies_df['rating'] >= min_rating_input) &
                                (movies_df['rating'] <= max_rating_input)]

    # Sort by predicted rating
    recommended_movies = filtered_movies.sort_values(by='predicted_rating', ascending=False).head(top_n)

    print("Top Recommended Movies:")
    print(recommended_movies[['title', 'rating']])

# Example usage:
genre_input = input("Enter a genre: ")
min_rating_input = float(input("Enter the minimum rating (0-5): "))
max_rating_input = float(input("Enter the maximum rating (0-5): "))

movies_df, genre_encoder = prepare_data(grouped_data)
recommend_movies_with_ann(grouped_data, genre_encoder, genre_input, min_rating_input, max_rating_input)


Enter a genre: Action|Crime|Thriller
Enter the minimum rating (0-5): 2.88
Enter the maximum rating (0-5): 3.97
Epoch 1/10
122/122 [==============================] - 1s 3ms/step - loss: 0.0230 - val_loss: 0.0085
Epoch 2/10
122/122 [==============================] - 0s 2ms/step - loss: 0.0030 - val_loss: 0.0011
Epoch 3/10
122/122 [==============================] - 0s 2ms/step - loss: 6.4848e-04 - val_loss: 7.0417e-04
Epoch 4/10
122/122 [==============================] - 0s 2ms/step - loss: 5.2033e-04 - val_loss: 6.4335e-04
Epoch 5/10
122/122 [==============================] - 0s 2ms/step - loss: 5.0128e-04 - val_loss: 6.2453e-04
Epoch 6/10
122/122 [==============================] - 0s 2ms/step - loss: 4.9793e-04 - val_loss: 6.3513e-04
Epoch 7/10
122/122 [==============================] - 0s 2ms/step - loss: 4.9250e-04 - val_loss: 6.1730e-04
Epoch 8/10
122/122 [==============================] - 0s 2ms/step - loss: 4.8853e-04 - val_loss: 6.2607e-04
Epoch 9/10
122/122 [=====================

**RECOMMENDATION MODEL WITH THE USER FEEDBACK**

In [ ]:
def recommend_movies_with_ann(movies_df, genre_encoder, genre_input, min_rating_input, max_rating_input, top_n=10):
    # Load pre-trained model
    model = build_model(len(genre_encoder.classes_))

    # Prepare input data
    X_genre = movies_df['genre_encoded'].values.reshape(-1, 1)
    X_rating = movies_df['rating_scaled'].values.reshape(-1, 1)
    y_train = movies_df['rating_scaled'].values

    train_model(model, [X_genre, X_rating], y_train)

    # Generate recommendations
    predictions = model.predict([X_genre, X_rating])
    movies_df['predicted_rating'] = predictions * 5  # Scale predictions back to original rating scale

    # Filter movies based on user input rating range and genre
    filtered_movies = movies_df[(movies_df['genres'].str.contains(genre_input)) &
                                (movies_df['rating'] >= min_rating_input) &
                                (movies_df['rating'] <= max_rating_input)]

    # Sort by predicted rating
    recommended_movies = filtered_movies.sort_values(by='predicted_rating', ascending=False).head(top_n)

    # Collect feedback for each recommended movie
    feedback = {}
    for _, movie_row in recommended_movies.iterrows():
        movie_title = movie_row['title']
        movie_feedback = input(f"Do you like the movie '{movie_title}'? (-1 for rare, 0 for no change, 1 for more): ")
        feedback[movie_title] = int(movie_feedback)

    # Adjust predicted ratings based on feedback
    recommended_movies['adjusted_rating'] = recommended_movies['predicted_rating']
    for movie_title, movie_feedback in feedback.items():
        feedback_multiplier = 1 + 0.2 * movie_feedback  # Adjust the multiplier as needed
        recommended_movies.loc[recommended_movies['title'] == movie_title, 'adjusted_rating'] *= feedback_multiplier

    # Sort by adjusted rating after incorporating feedback
    recommended_movies_with_feedback = recommended_movies.sort_values(by='adjusted_rating', ascending=False).head(top_n)

    print("Top Recommended Movies with Feedback:")
    print(recommended_movies_with_feedback[['title', 'rating']])

# Example usage:
genre_input = input("Enter a genre: ")
min_rating_input = float(input("Enter the minimum rating (0-5): "))
max_rating_input = float(input("Enter the maximum rating (0-5): "))

movies_df, genre_encoder = prepare_data(grouped_data)
recommend_movies_with_ann(movies_df, genre_encoder, genre_input, min_rating_input, max_rating_input)


Enter a genre: Action|Crime|Thriller
Enter the minimum rating (0-5): 2.88
Enter the maximum rating (0-5): 3.97
Epoch 1/10
122/122 [==============================] - 1s 3ms/step - loss: 0.0240 - val_loss: 0.0091
Epoch 2/10
122/122 [==============================] - 0s 2ms/step - loss: 0.0031 - val_loss: 0.0012
Epoch 3/10
122/122 [==============================] - 0s 2ms/step - loss: 6.4192e-04 - val_loss: 6.9927e-04
Epoch 4/10
122/122 [==============================] - 0s 2ms/step - loss: 5.2049e-04 - val_loss: 6.4364e-04
Epoch 5/10
122/122 [==============================] - 0s 2ms/step - loss: 5.0233e-04 - val_loss: 6.2456e-04
Epoch 6/10
122/122 [==============================] - 0s 2ms/step - loss: 4.9963e-04 - val_loss: 6.2542e-04
Epoch 7/10
122/122 [==============================] - 0s 2ms/step - loss: 4.9065e-04 - val_loss: 6.0999e-04
Epoch 8/10
122/122 [==============================] - 0s 2ms/step - loss: 4.8404e-04 - val_loss: 6.0002e-04
Epoch 9/10
122/122 [=====================

In [ ]:
def recommend_movies_with_ann(movies_df, genre_encoder, genre_input, min_rating_input, max_rating_input, top_n=10):
    # Load pre-trained model
    model = build_model(len(genre_encoder.classes_))

    # Prepare input data
    X_genre = movies_df['genre_encoded'].values.reshape(-1, 1)
    X_rating = movies_df['rating_scaled'].values.reshape(-1, 1)
    y_train = movies_df['rating_scaled'].values

    train_model(model, [X_genre, X_rating], y_train)

    # Generate recommendations
    predictions = model.predict([X_genre, X_rating])
    movies_df['predicted_rating'] = predictions * 5  # Scale predictions back to original rating scale

    # Filter movies based on user input rating range and genre
    filtered_movies = movies_df[(movies_df['genres'].str.contains(genre_input)) &
                                (movies_df['rating'] >= min_rating_input) &
                                (movies_df['rating'] <= max_rating_input)]

    # Sort by predicted rating
    recommended_movies = filtered_movies.sort_values(by='predicted_rating', ascending=False).head(top_n)

    # Collect feedback for each recommended movie
    feedback = {}
    for _, movie_row in recommended_movies.iterrows():
        movie_title = movie_row['title']
        movie_feedback = input(f"Do you like the movie '{movie_title}'? (-1 for rare, 0 for no change, 1 for more): ")
        feedback[movie_title] = int(movie_feedback)

    # Remove movies with negative feedback
    recommended_movies = recommended_movies[recommended_movies['title'].map(lambda x: feedback.get(x, 0) != -1)]

    # Adjust predicted ratings based on feedback
    recommended_movies['adjusted_rating'] = recommended_movies['predicted_rating']
    for movie_title, movie_feedback in feedback.items():
        feedback_multiplier = 1 + 0.2 * movie_feedback  # Adjust the multiplier as needed
        recommended_movies.loc[recommended_movies['title'] == movie_title, 'adjusted_rating'] *= feedback_multiplier

    # Sort by adjusted rating after incorporating feedback
    recommended_movies_with_feedback = recommended_movies.sort_values(by='adjusted_rating', ascending=False).head(top_n)

    print("Top Recommended Movies with Feedback:")
    print(recommended_movies_with_feedback[['title', 'rating']])

# Example usage:
genre_input = input("Enter a genre: ")
min_rating_input = float(input("Enter the minimum rating (0-5): "))
max_rating_input = float(input("Enter the maximum rating (0-5): "))

movies_df, genre_encoder = prepare_data(grouped_data)
recommend_movies_with_ann(movies_df, genre_encoder, genre_input, min_rating_input, max_rating_input)



Enter a genre: Action|Crime|Thriller
Enter the minimum rating (0-5): 2.43
Enter the maximum rating (0-5): 3.22
Epoch 1/10
122/122 [==============================] - 1s 3ms/step - loss: 0.0218 - val_loss: 0.0082
Epoch 2/10
122/122 [==============================] - 0s 2ms/step - loss: 0.0030 - val_loss: 0.0011
Epoch 3/10
122/122 [==============================] - 0s 2ms/step - loss: 6.3527e-04 - val_loss: 6.7991e-04
Epoch 4/10
122/122 [==============================] - 0s 2ms/step - loss: 5.1261e-04 - val_loss: 6.3330e-04
Epoch 5/10
122/122 [==============================] - 0s 2ms/step - loss: 4.9764e-04 - val_loss: 6.2062e-04
Epoch 6/10
122/122 [==============================] - 0s 2ms/step - loss: 4.9041e-04 - val_loss: 6.2109e-04
Epoch 7/10
122/122 [==============================] - 0s 2ms/step - loss: 4.8476e-04 - val_loss: 6.2147e-04
Epoch 8/10
122/122 [==============================] - 0s 2ms/step - loss: 4.8089e-04 - val_loss: 6.1998e-04
Epoch 9/10
122/122 [=====================

**RECOMMENDING ATLEAT 10 MOVIES IF AVAILABLE IN DATASET**

In [ ]:
def recommend_movies_with_ann(movies_df, genre_encoder, genre_input, min_rating_input, max_rating_input, top_n=10):
    # Load pre-trained model
    model = build_model(len(genre_encoder.classes_))

    # Prepare input data
    X_genre = movies_df['genre_encoded'].values.reshape(-1, 1)
    X_rating = movies_df['rating_scaled'].values.reshape(-1, 1)
    y_train = movies_df['rating_scaled'].values

    train_model(model, [X_genre, X_rating], y_train)

    # Generate recommendations
    predictions = model.predict([X_genre, X_rating])
    movies_df['predicted_rating'] = predictions * 5  # Scale predictions back to original rating scale

    # Filter movies based on user input rating range and genre
    filtered_movies = movies_df[(movies_df['genres'].str.contains(genre_input)) &
                                (movies_df['rating'] >= min_rating_input) &
                                (movies_df['rating'] <= max_rating_input)]

    # Sort by predicted rating
    recommended_movies = filtered_movies.sort_values(by='predicted_rating', ascending=False)

    # Collect feedback for the top N recommended movies
    feedback = {}
    count = 0
    for _, movie_row in recommended_movies.iterrows():
        if count >= top_n:
            break
        movie_title = movie_row['title']
        movie_feedback = input(f"Do you like the movie '{movie_title}'? (-1 for rare, 0 for no change, 1 for more): ")
        feedback[movie_title] = int(movie_feedback)
        count += 1

    # Get the list of movies with feedback of -1
    negative_feedback_movies = [movie_title for movie_title, movie_feedback in feedback.items() if movie_feedback == -1]

    # Find similar movies for movies with -1 feedback
    similar_movies = movies_df[movies_df['title'].isin(negative_feedback_movies)]

    # Filter similar movies based on genre and rating range
    similar_movies = similar_movies[(similar_movies['genres'].str.contains(genre_input)) &
                                    (similar_movies['rating'] >= min_rating_input) &
                                    (similar_movies['rating'] <= max_rating_input)]

    # Concatenate recommended movies and similar movies, dropping duplicates
    recommended_movies = pd.concat([recommended_movies, similar_movies]).drop_duplicates(subset='title')

    # Adjust predicted ratings based on feedback
    recommended_movies['adjusted_rating'] = recommended_movies['predicted_rating']
    for movie_title, movie_feedback in feedback.items():
        feedback_multiplier = 1 + 0.2 * movie_feedback  # Adjust the multiplier as needed
        recommended_movies.loc[recommended_movies['title'] == movie_title, 'adjusted_rating'] *= feedback_multiplier

    # Sort by adjusted rating after incorporating feedback
    recommended_movies_with_feedback = recommended_movies.sort_values(by='adjusted_rating', ascending=False).head(top_n)

    print("Top Recommended Movies with Feedback:")
    print(recommended_movies_with_feedback[['title', 'rating']])

# Example usage:
genre_input = input("Enter a genre: ")
min_rating_input = float(input("Enter the minimum rating (0-5): "))
max_rating_input = float(input("Enter the maximum rating (0-5): "))

movies_df, genre_encoder = prepare_data(grouped_data)
recommend_movies_with_ann(movies_df, genre_encoder, genre_input, min_rating_input, max_rating_input)
1

Enter a genre: Action|Crime|Thriller
Enter the minimum rating (0-5): 0
Enter the maximum rating (0-5): 5
Epoch 1/10
122/122 [==============================] - 1s 2ms/step - loss: 0.0224 - val_loss: 0.0096
Epoch 2/10
122/122 [==============================] - 0s 2ms/step - loss: 0.0035 - val_loss: 0.0013
Epoch 3/10
122/122 [==============================] - 0s 2ms/step - loss: 6.6976e-04 - val_loss: 6.9472e-04
Epoch 4/10
122/122 [==============================] - 0s 2ms/step - loss: 5.2024e-04 - val_loss: 6.4206e-04
Epoch 5/10
122/122 [==============================] - 0s 2ms/step - loss: 5.0201e-04 - val_loss: 6.2532e-04
Epoch 6/10
122/122 [==============================] - 0s 2ms/step - loss: 4.9395e-04 - val_loss: 6.1914e-04
Epoch 7/10
122/122 [==============================] - 0s 2ms/step - loss: 4.8256e-04 - val_loss: 6.0843e-04
Epoch 8/10
122/122 [==============================] - 0s 2ms/step - loss: 4.7038e-04 - val_loss: 6.0168e-04
Epoch 9/10
122/122 [===========================

In [ ]:
import pandas as pd

def recommend_movies_with_ann(movies_df, genre_encoder, genre_input, min_rating_input, max_rating_input, user_id, top_n=10):
    # Load pre-trained model
    model = build_model(len(genre_encoder.classes_))

    # Prepare input data
    X_genre = movies_df['genre_encoded'].values.reshape(-1, 1)
    X_rating = movies_df['rating_scaled'].values.reshape(-1, 1)
    y_train = movies_df['rating_scaled'].values

    train_model(model, [X_genre, X_rating], y_train)

    # Generate recommendations
    predictions = model.predict([X_genre, X_rating])
    movies_df['predicted_rating'] = predictions * 5  # Scale predictions back to original rating scale

    # Filter movies based on user input rating range and genre
    filtered_movies = movies_df[(movies_df['genres'].str.contains(genre_input)) &
                                (movies_df['rating'] >= min_rating_input) &
                                (movies_df['rating'] <= max_rating_input)]

    # Sort by predicted rating
    recommended_movies = filtered_movies.sort_values(by='predicted_rating', ascending=False)

    # Collect feedback for the top N recommended movies
    feedback = {}
    count = 0
    for _, movie_row in recommended_movies.iterrows():
        if count >= top_n:
            break
        movie_title = movie_row['title']
        movie_feedback = input(f"Do you like the movie '{movie_title}'? (-1 for rare, 0 for no change, 1 for more): ")
        feedback[movie_title] = int(movie_feedback)
        count += 1

    # Get the list of movies with feedback of -1
    negative_feedback_movies = [movie_title for movie_title, movie_feedback in feedback.items() if movie_feedback == -1]

    # Find similar movies for movies with -1 feedback
    similar_movies = movies_df[movies_df['title'].isin(negative_feedback_movies)]

    # Filter similar movies based on genre and rating range
    similar_movies = similar_movies[(similar_movies['genres'].str.contains(genre_input)) &
                                    (similar_movies['rating'] >= min_rating_input) &
                                    (similar_movies['rating'] <= max_rating_input)]

    # Concatenate recommended movies and similar movies, dropping duplicates
    recommended_movies = pd.concat([recommended_movies, similar_movies]).drop_duplicates(subset='title')

    # Adjust predicted ratings based on feedback
    recommended_movies['adjusted_rating'] = recommended_movies['predicted_rating']
    for movie_title, movie_feedback in feedback.items():
        feedback_multiplier = 1 + 0.2 * movie_feedback  # Adjust the multiplier as needed
        recommended_movies.loc[recommended_movies['title'] == movie_title, 'adjusted_rating'] *= feedback_multiplier

    # Sort by adjusted rating after incorporating feedback
    recommended_movies_with_feedback = recommended_movies.sort_values(by='adjusted_rating', ascending=False).head(top_n)

    # Store recommendations and feedback in a CSV file
    filename = f"recommendations_user_{user_id}.csv"
    recommended_movies_with_feedback.to_csv(filename, index=False)

    print("Top Recommended Movies with Feedback:")
    print(recommended_movies_with_feedback[['title', 'rating']])

# Example usage:
user_id = input("Enter your user ID: ")
genre_input = input("Enter a genre: ")
min_rating_input = float(input("Enter the minimum rating (0-5): "))
max_rating_input = float(input("Enter the maximum rating (0-5): "))

movies_df, genre_encoder = prepare_data(grouped_data)
recommend_movies_with_ann(movies_df, genre_encoder, genre_input, min_rating_input, max_rating_input, user_id)


Enter your user ID: 345788
Enter a genre: Horror|Mystery
Enter the minimum rating (0-5): 0
Enter the maximum rating (0-5): 5
Epoch 1/10
122/122 [==============================] - 1s 3ms/step - loss: 0.0231 - val_loss: 0.0101
Epoch 2/10
122/122 [==============================] - 0s 2ms/step - loss: 0.0038 - val_loss: 0.0013
Epoch 3/10
122/122 [==============================] - 0s 2ms/step - loss: 6.9149e-04 - val_loss: 6.9597e-04
Epoch 4/10
122/122 [==============================] - 0s 1ms/step - loss: 5.1971e-04 - val_loss: 6.3472e-04
Epoch 5/10
122/122 [==============================] - 0s 2ms/step - loss: 4.9972e-04 - val_loss: 6.2302e-04
Epoch 6/10
122/122 [==============================] - 0s 1ms/step - loss: 4.9310e-04 - val_loss: 6.2038e-04
Epoch 7/10
122/122 [==============================] - 0s 1ms/step - loss: 4.8634e-04 - val_loss: 6.2067e-04
Epoch 8/10
122/122 [==============================] - 0s 2ms/step - loss: 4.8294e-04 - val_loss: 6.2779e-04
Epoch 9/10
304/304 [=======